# Imports and dataframes read

In [12]:
import pandas as pd
import numpy as np
import csv as cv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# read csv file into dataframe
df = pd.read_csv('new_anime.csv')
print(df.shape)

(4375, 8)


# Processing csv file

In [14]:
data = range(len(df))

for i in data:
    if df.loc[i]['VAs'] == ' \'\'':
        df = df.drop(labels=i,axis=0)

df.shape

(3568, 8)

In [15]:
def create_soup(x):
    result = x['Anime Title']

    if x['Studio'] != 'n/a':
        result += ' ' + x['Studio']
    if x['Source Material'] != 'n/a':
        result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a':
        result += ' ' + x['Genre']
    if x['VAs'] != 'n/a':
        result += ' ' + x['VAs']
    
    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    return result

df['soup'] = df.apply(create_soup,axis = 1)

#df = df.drop(['Rating', 'Studio', 'Source Material', 'Demographic', 'Genre', 'VAs'], axis=1)

df.to_csv("anime_soup.csv", index=None)

print(df['soup'])

0       KonoSubarashiiSekainiShukufukuwo!2 StudioDeen ...
1       Kobayashi-sanChinoMaidDragon KyotoAnimation Ma...
2       YoujoSenki Nut Lightnovel Action Isekai Milita...
3       Masamune-kunnoRevenge SILVERLINK. Manga Comedy...
4       AonoExorcist:KyotoFujouou-hen A-1Pictures Mang...
                              ...                        
4355    KaitouJoker4thSeason Shin-EiAnimation Manga Ad...
4356    BishoujoYuugiUnitCraneGameGirlsGalaxy Kyotoma ...
4357    3-NenD-GumiGlassnoKamen DLE Manga Comedy Parod...
4358    Heybot! BandaiNamcoPictures Original Comedy Sc...
4360    KamigaminoKi Sotsu Original Comedy Supernatura...
Name: soup, Length: 3568, dtype: object


In [16]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['soup'])
count_matrix.shape

(3568, 28261)

In [17]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

print(cosine_sim2)

df = df.reset_index()
# indices = pd.Series(df.index, index=df['Id'])

# print(indices)

[[1.         0.07302967 0.10741723 ... 0.09759001 0.08377078 0.02663118]
 [0.07302967 1.         0.         ... 0.1069045  0.         0.        ]
 [0.10741723 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.09759001 0.1069045  0.         ... 1.         0.12262787 0.03898406]
 [0.08377078 0.         0.         ... 0.12262787 1.         0.06692745]
 [0.02663118 0.         0.         ... 0.03898406 0.06692745 1.        ]]


In [36]:
def get_recs(id, cosine_sim = cosine_sim2):
    # get the index of the anime that matches the id
    id2 = '\'' + str(id) + '\''
    idx = (df.loc[df['Id'] == id2].index[0])

    # find sim score of the inputed anime with all other animes
    sim_scores = list(enumerate(cosine_sim[idx]))

    # sort based on sim scores desc
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # get recommended animes index
    anime_indices = [i[0] for i in sim_scores]
    for i in range(len(sim_scores)):
        if i > 10:
            break
        else:
            print(df['Anime Title'].iloc[anime_indices[i]])

    # return df['Anime Title'].iloc[anime_indices]

In [38]:
get_recs(21, cosine_sim2)

'OnePiece'
'GetterRoboGo'
'BoukenOuBeetExcellion'
'Dokidoki♡Densetsu:MahoujinGuruguru'
'DragonBallKai'
'KishinDoujiZenki'
'NogsaegjeonchaHamos'
'Croket!'
'KuusouKagakuSekaiGulliverBoy'
'AsobotSenkiGokuu'
'Toriko'
